In [1]:
!pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd


In [2]:
file_path = "spotify-2023.csv"

In [7]:
df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "nelgiriyewithana/top-spotify-songs-2023",
        file_path,
        pandas_kwargs={
            "low_memory": False,
            "encoding": "latin1"  # Try 'latin1' to handle special characters
        }
    )
print("Dataset loaded successfully.")

<ipython-input-7-824f33057944>:1: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


Dataset loaded successfully.


In [8]:
print("Shape of the dataset:", df.shape)
print("Columns in the dataset:", df.columns.tolist())



Shape of the dataset: (953, 24)
Columns in the dataset: ['track_name', 'artist(s)_name', 'artist_count', 'released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm', 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']


In [37]:
import pandas as pd
print("Dataset Info:")
print(df.info())
print("\nFirst 5 Rows:")
print(df.head())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            952 non-null    object 
 1   artist(s)_name        952 non-null    object 
 2   artist_count          952 non-null    int64  
 3   released_year         952 non-null    int64  
 4   released_month        952 non-null    int64  
 5   released_day          952 non-null    int64  
 6   in_spotify_playlists  952 non-null    int64  
 7   in_spotify_charts     952 non-null    int64  
 8   streams               952 non-null    float64
 9   in_apple_playlists    952 non-null    int64  
 10  in_apple_charts       952 non-null    int64  
 11  in_deezer_playlists   952 non-null    object 
 12  in_deezer_charts      952 non-null    int64  
 13  in_shazam_charts      952 non-null    object 
 14  bpm                   952 non-null    int64  
 15  key           

In [23]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

numerical_features = [
    'danceability_%','energy_%'
]
df_features = df[numerical_features]

scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_features),
    columns=numerical_features,
    index=df.index
)
knn = NearestNeighbors(n_neighbors=6, algorithm='auto', metric='euclidean')
knn.fit(df_scaled)

print("KNN model trained successfully with features:", numerical_features)

KNN model trained successfully with features: ['danceability_%', 'energy_%']


In [24]:
def recommend_songs(song_name: str, df: pd.DataFrame, df_scaled: pd.DataFrame, knn: NearestNeighbors, top_n: int = 5):
    song_indices = df[df['track_name'].str.lower() == song_name.lower()].index

    if len(song_indices) == 0:
        return "Song not found in the dataset. Please check the name and try again."

    #first matching song's index
    song_index = song_indices[0]

    # Get scaled features for the song
    song_features = df_scaled.iloc[song_index].values.reshape(1, -1)

    distances, indices = knn.kneighbors(song_features)

    recommended_indices = indices[0][1:top_n+1]

    output_columns = ['track_name', 'artist(s)_name'] + [
        'danceability_%', 'energy_%','valence_%' ]
    recommendations = df.iloc[recommended_indices][output_columns]


    print(f"Recommendations for '{song_name}':")
    return recommendations
if __name__ == "__main__":
    song_input = "Blinding Lights"
    recommendations = recommend_songs(song_input, df, df_scaled, knn)
    print(recommendations)

Recommendations for 'Blinding Lights':
                       track_name            artist(s)_name  danceability_%  \
395                    Space Song               Beach House              51   
450           Underneath the Tree            Kelly Clarkson              51   
772  Cooped Up (with Roddy Ricch)  Post Malone, Roddy Ricch              50   
451                 Feliz Navidad           Josï¿½ï¿½ Felic              50   
304               SORRY NOT SORRY        Tyler, The Creator              48   

     energy_%  valence_%  
395        79         62  
450        81         69  
772        78         39  
451        82         96  
304        80         50  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [25]:
song_input = "Feliz Navidad"
recommendations = recommend_songs(song_input, df, df_scaled, knn)
print(recommendations)

Recommendations for 'Feliz Navidad':
              track_name       artist(s)_name  danceability_%  energy_%  \
793    House Of Memories  Panic! At The Disco              51        82   
450  Underneath the Tree       Kelly Clarkson              51        81   
55       Blinding Lights           The Weeknd              50        80   
768      United In Grief       Kendrick Lamar              52        83   
304      SORRY NOT SORRY   Tyler, The Creator              48        80   

     valence_%  
793         48  
450         69  
55          38  
768         32  
304         50  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


**NEXT IS DATASPLIT AND USE DIFFERENT DATASET TO WORK ON**

In [26]:
import pandas as pd
import numpy as np

n = len(df)
split_indices = [n//3, 2*n//3]

# Split the dataset
df_part1, df_part2, df_part3 = np.split(df, split_indices)

# Print shapes
print("Shape of Part 1:", df_part1.shape)
print("Shape of Part 2:", df_part2.shape)
print("Shape of Part 3:", df_part3.shape)

Shape of Part 1: (317, 24)
Shape of Part 2: (317, 24)
Shape of Part 3: (318, 24)


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [31]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

def master_recommend_songs(song_name: str, main_df: pd.DataFrame, split_df: pd.DataFrame, top_n: int = 5, features: list = ['danceability_%', 'energy_%', 'valence_%']):

    song_indices = main_df[main_df['track_name'].str.lower() == song_name.lower()].index
    if len(song_indices) == 0:
        return f"Song '{song_name}' not found in main dataset."

    song_index = song_indices[0]

    df_features = split_df[features]
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(
        scaler.fit_transform(df_features),
        columns=features,
        index=split_df.index
    )
    knn = NearestNeighbors(n_neighbors=top_n+1, algorithm='auto', metric='euclidean')
    knn.fit(df_scaled)

    main_features = main_df.iloc[song_index][features].values.reshape(1, -1)
    song_features = scaler.transform(main_features)

    distances, indices = knn.kneighbors(song_features)
    recommended_indices = indices[0][:top_n]

    output_columns = ['track_name', 'artist(s)_name'] + features
    recommendations = split_df.iloc[recommended_indices][output_columns]

    print(f"Recommendations for '{song_name}' from split ({len(split_df)} rows):")
    return recommendations

In [32]:

song_input = "Blinding Lights"
recommendations1 = master_recommend_songs(song_input, df, df_part1)
print(recommendations1)

Recommendations for 'Blinding Lights' from split (317 rows):
          track_name      artist(s)_name  danceability_%  energy_%  valence_%
55   Blinding Lights          The Weeknd              50        80         38
255         Curtains          Ed Sheeran              50        76         44
116           Tattoo              Loreen              55        78         30
304  SORRY NOT SORRY  Tyler, The Creator              48        80         50
243      Unstoppable                 Sia              47        78         27


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [33]:

song_input = "Blinding Lights"
recommendations2 = master_recommend_songs(song_input, df, df_part2)
print(recommendations2)

Recommendations for 'Blinding Lights' from split (317 rows):
                             track_name              artist(s)_name  \
479          Wild Flower (with youjeen)                          RM   
394                 Escapism. - Sped Up             RAYE, 070 Shake   
347  Man Made A Bar (feat. Eric Church)  Morgan Wallen, Eric Church   
473                                 F2F                         SZA   
511                      Less Than Zero                  The Weeknd   

     danceability_%  energy_%  valence_%  
479              49        77         42  
394              44        77         38  
347              50        76         49  
473              53        74         47  
511              53        79         50  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [34]:

song_input = "Blinding Lights"
recommendations3 = master_recommend_songs(song_input, df, df_part3)
print(recommendations3)

Recommendations for 'Blinding Lights' from split (318 rows):
                       track_name            artist(s)_name  danceability_%  \
772  Cooped Up (with Roddy Ricch)  Post Malone, Roddy Ricch              50   
768               United In Grief            Kendrick Lamar              52   
818                     SPACE MAN                 Sam Ryder              47   
817    10 Things I Hate About You                 Leah Kate              54   
684                   Alien Blues                  Vundabar              47   

     energy_%  valence_%  
772        78         39  
768        83         32  
818        83         33  
817        79         45  
684        76         44  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
